<H3>Using craigslist for rental details to cluster with neighborhood venus<H3>

In [1]:
#required imports for the project!!
import numpy as np 
import pandas as pd

# library to scrap the web page 
from bs4 import BeautifulSoup

# library to handle JSON files
import json

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

# matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Libraries imported.


In [2]:
#using craigslist to get rental data of San Francisco up to 50 listings!!
from craigslist import CraigslistHousing
cl = CraigslistHousing(site='sfbay', area='sfc', category='apa')
results = cl.get_results(sort_by='newest', geotagged=True, limit=100)

In [4]:
#creating data frame of the rental data!!
rental_data = pd.DataFrame(results)
rental_data['geotag'] = rental_data['geotag'].map(lambda x: str(x).lstrip('(').rstrip(')'))
rental_data['Latitude'], rental_data['Longitude'] = rental_data['geotag'].str.split(',', 1).str
rental_data['Latitude'] = rental_data['Latitude'].astype(float)
rental_data['Longitude'] = rental_data['Longitude'].astype(float)
rental_data.drop(['geotag','last_updated','has_image','datetime','repost_of'],1,inplace=True)
rental_data.drop_duplicates(subset='id',keep='first',inplace=True)
rental_data

,id,name,url,price,where,Latitude,Longitude
0,7032990766,"Prime Location, Amenities, Pet Friendly, Wooft...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$4190,SOMA / south beach,37.786588,-122.401596
1,7047709792,"1 BR in 3BR, 2 BA Apartment",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$1850,lower pac hts,37.785600,-122.435800
2,7040514449,Remodeled 1 Bedroom 1 Bath- Furnished- laundr...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$1800,portola district,37.719000,-122.409600
3,7047709931,Spectacular Panoramic City Skyline&Bay views ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$5395,castro / upper market,37.757340,-122.445891
4,7047709758,Spectacular Panoramic City Skyline&Bay views ...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$4895,castro / upper market,37.757340,-122.445891
...,...,...,...,...,...,...,...
95,7044873936,Beautiful HUGE Two Bedroom Apartment close to ...,https://sfbay.craigslist.org/sfc/apa/d/beautif...,$4200,russian hill,37.792507,-122.420076
96,7044873315,Exceptional 2 Bedroom Apartment in Russian Hill,https://sfbay.craigslist.org/sfc/apa/d/excepti...,$4600,russian hill,37.792507,-122.420076
97,7044873557,DELUXE 1 STUDIO PREWAR APARTMENT NEAR LAFAYETT...,https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$2500,pacific heights,37.791064,-122.424295
98,7047646293,"Remodeled Single Family Home with Garage, Yard...",https://sfbay.craigslist.org/sfc/apa/d/san-fra...,$11500,cole valley / ashbury hts,37.762299,-122.451070


In [5]:
#geolocation of San Francisco
address = 'San Francisco'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7792808, -122.4192363.


In [6]:
#creating map of San Francisco!!
map_sfo = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lati, lngi, label in zip(rental_data['Latitude'], rental_data['Longitude'], rental_data['where']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lati, lngi],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sfo)  
    
map_sfo

In [7]:
#foursquare credentials
CLIENT_ID = 'B1TIJO5AFFZF4Y4BL4OUTN1VMWAFZGVN2JF2YOKOZQTXEOJA' # your Foursquare ID
CLIENT_SECRET = 'UPCL15AJXRZEIHKX20PHXAHZQFY0D2352GE5KCGBU2LHTISY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B1TIJO5AFFZF4Y4BL4OUTN1VMWAFZGVN2JF2YOKOZQTXEOJA
CLIENT_SECRET:UPCL15AJXRZEIHKX20PHXAHZQFY0D2352GE5KCGBU2LHTISY


In [8]:
#function to get foursquare data for all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
#get the required data 
sfo_venues = getNearbyVenues(names=rental_data['where'],
                             latitudes=rental_data['Latitude'],
                             longitudes=rental_data['Longitude']
                            )


In [10]:
#data frame containing the san francisco venus data!!
sfo_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,SOMA / south beach,37.786588,-122.401596,San Francisco Museum of Modern Art,37.785894,-122.400897,Art Museum
1,SOMA / south beach,37.786588,-122.401596,Flatiron Wine and Spirits,37.788039,-122.401466,Wine Shop
2,SOMA / south beach,37.786588,-122.401596,SFMOMA Rooftop Sculpture Garden,37.786336,-122.400404,Art Museum
3,SOMA / south beach,37.786588,-122.401596,The St. Regis San Francisco,37.786130,-122.401468,Hotel
4,SOMA / south beach,37.786588,-122.401596,Museum of the African Diaspora,37.786590,-122.401680,Museum
